## **Installing dependencies** 

In [ ]:
import torch
torch.cuda.empty_cache()

In [2]:
# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need sentencepiece for the llama2 slow tokenizer (not necessary for mistral)
!pip install sentencepiece

# we need einops, used by falcon-7b, llama-2 mistral etc
# einops (einsteinops) is used to simplify tensorops by making them readable
!pip install -q -U einops

# we need to install datasets for our training dataset
!pip install -q datasets


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch

def print_gpu_memory_usage():
    # Assuming you have at least one CUDA-capable GPU and want to check the first one
    t = torch.cuda.get_device_properties(0).total_memory
    r = torch.cuda.memory_reserved(0) 
    a = torch.cuda.memory_allocated(0)
    f = r-a  # free inside reserved

    print(f"Total GPU Memory: {t * 1e-9:.2f} GB")
    print(f"Reserved Memory: {r * 1e-9:.2f} GB")
    print(f"Allocated Memory: {a * 1e-9:.2f} GB")
    print(f"Free (inside reserved) Memory: {f * 1e-9:.2f} GB")

# Then, you can call this function at the end of each cell where you want to check the GPU memory usage
print_gpu_memory_usage()


Total GPU Memory: 34.07 GB
Reserved Memory: 0.00 GB
Allocated Memory: 0.00 GB
Free (inside reserved) Memory: 0.00 GB


## **Loading the model** 

In [2]:
# The model that we want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# The instruction dataset to use found on HuggingFace
dataset_name = "MaamarM/SAS_training"

# Fine-tuned model name
new_model = "Mistral-7B-SAS"

# Output directory where the model predictions and checkpoints will be stored
#This is to push our fine tuned model in the Hugging Face platform
output_dir = "./Mistral_Instruct_SAS"

# Number of training epochs
num_train_epochs = 8

In [3]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

# Load the tokenizer from the model (mistral)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

2024-04-10 15:15:54.711886: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-10 15:15:54.758171: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
print_gpu_memory_usage()

Total GPU Memory: 34.07 GB
Reserved Memory: 5.19 GB
Allocated Memory: 5.03 GB
Free (inside reserved) Memory: 0.16 GB


## **Testing on some prompts** 

In [4]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
prompt = "Create a new data table called (sports) which has only three columns from the (cars) data set: make, type, and msrp. Keep only those observations where type is equal to (Sports) and msrp is greater than $100,000. Then, create another data table called (suv) which has the same three columns. Keep only those observations where type is equal to (SUV) and msrp is greater than $60,000. Finally, create a new data table called (expensive) by row-binding (sports) and (suv). Print (expensive) to see the results."

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/home/zealous_easley/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[INST] Create a new data table called (sports) which has only three columns from the (cars) data set: make, type, and msrp. Keep only those observations where type is equal to (Sports) and msrp is greater than $100,000. Then, create another data table called (suv) which has the same three columns. Keep only those observations where type is equal to (SUV) and msrp is greater than $60,000. Finally, create a new data table called (expensive) by row-binding (sports) and (suv). Print (expensive) to see the results. [/INST] To create the new data tables `sports` and `suv` based on the given conditions from the `cars` dataset, you can use the following code:

```r
# Create sports data table
sports <- subset(cars, type == "Sports" & msrp > 100000) %>%
  select(make, type, msrp)

# Create suv data table
suv <- subset(cars, type == "SUV" & msrp > 60000) %>%
  select(make, type, msrp)

# Create expensive data table by row-binding sports and suv
expensive <- rbind(sports, suv)

# Print the expensi

In [15]:
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

parquet_file = r'test_data.parquet'
df=pd.read_parquet(parquet_file, engine = 'pyarrow')
df

,content,Annotation
4829,*SAS code;\nPROC REG DATA= REG_SERIES; MODEL y...,Perform a linear regression analysis on the da...
5081,%let name=sct4;\n\n/* \nSet your current-worki...,"Set a variable named ""name"" to ""sct4"", create ..."
5011,options bufsize=32768 pagesize=10000;\n\ndata ...,"Define the options ""bufsize"" and ""pagesize"" to..."
4625,"LIBNAME X ""/folders/myfolders/X"";\ndata brfss_...","""Set the library location to '/folders/myfolde..."
3838,*Ex15_macro_part_of_SAS_statement.sas;\noption...,Please convert the following SAS programming l...
...,...,...
3819,Parent child kinship problem chain\n\n WPS an...,"Create an intermediate dataset named ""int"" by ..."
10089,/*********************************************...,Create an LLM model that can replicate the fun...
8740,Cross Platform Unzip Medicare Point of Sevice ...,Instructions for an LLM model to match the giv...
7831,"%include ""HIDI_init.sas"" ; /* File path remove...","""Include the following SAS files in the order ..."


In [29]:
results = []
def generate_script(instruction):
    result = pipe(f"[INST] {instruction} [/INST]")
    generated_text = result[0]['generated_text']
    cleaned_output = generated_text.replace(f"[INST] {instruction} [/INST]", "").strip()
    return instruction, cleaned_output

# Use ThreadPoolExecutor to parallelize script generation
with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust max_workers based on your hardware
    future_to_inst = {executor.submit(generate_script, inst): inst for inst in df['Annotation'].iloc[:100]}
    for future in tqdm(as_completed(future_to_inst), total=len(future_to_inst), desc="Generating Scripts"):
        instruction, script = future.result()
        results.append({"instruction": instruction, "generated_script": script})

# Convert results to DataFrame and save to Parquet
df_results = pd.DataFrame(results)
df_results.to_parquet('generated_scripts.parquet')

Generating Scripts:   4%|▍         | 4/100 [05:54<2:02:28, 76.55s/it] /home/zealous_easley/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Generating Scripts: 100%|██████████| 100/100 [2:57:47<00:00, 106.68s/it] 


In [30]:
parquet_file = r'generated_scripts.parquet'
df=pd.read_parquet(parquet_file, engine = 'pyarrow')
df

,instruction,generated_script
0,"""Set the library location to '/folders/myfolde...","To set the SAS library location, use the follo..."
1,Please convert the following SAS programming l...,To create an LLM model instruction for the giv...
2,Rename a file located in a specific path using...,To rename the files using the given file names...
3,Perform a linear regression analysis on the da...,"To perform a linear regression analysis, save ..."
4,"Set a variable named ""name"" to ""sct4"", create ...","```scss:scss\n\nname <- ""sct4""\nfile.edit(""."",..."
...,...,...
95,"Set the library named ""es"" to the directory ""/...","To accomplish the tasks you described, you can..."
96,Create SAS informats for missing values using ...,To create SAS informats for missing values usi...
97,Create an LLM model that can replicate the fun...,"To create an LLM model named ""lscmd"" version 1..."
98,Create a function that creates an Application ...,Here's a VBA function that creates an Applicat...


In [32]:
df['generated_script'].iloc[0]

'To set the SAS library location, use the following command:\n\n```sas\nOPTIONS NOXWAIT NOXSYNC;\nLIBNAME lib "folders/myfolders/X" SERVER;\n```\n\nReplace "X" with the name of your library.\n\nNow, load the dataset \'Chap6_1\' from the library \'X\' into a new dataset named \'brfss_a\':\n\n```sas\nDATA brfss_a;\n    SET lib.Chap6_1;\nRUN;\n```\n\nFinally, use the \'PROC CONTENTS\' procedure to display variable information for the dataset \'brfss_a\':\n\n```sas\nPROC CONTENTS DATA=brfss_a;\nRUN;\n```\n\nThis will display the variable names and their corresponding variable numbers in the SAS log.'

## **Fine tuning sur SAS**

In [7]:
from datasets import load_dataset

data_files = {"train":"formatted_training.jsonl"}

# Load the dataset
dataset = load_dataset(dataset_name, data_files = data_files , split="train[:50%]")

In [8]:
print_gpu_memory_usage()

Total GPU Memory: 34.07 GB
Reserved Memory: 10.23 GB
Allocated Memory: 10.04 GB
Free (inside reserved) Memory: 0.19 GB


In [5]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"


In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,      # uses the number of epochs earlier
    per_device_train_batch_size=1,          # 4 seems reasonable
    gradient_accumulation_steps=1,          # 2 is fine, as we're a small batch
    optim="paged_adamw_32bit",              # default optimizer
    save_steps=0,                           # we're not gonna save
    logging_steps=10,                       # same value as used by Meta
    learning_rate=2e-4,                     # standard learning rate
    weight_decay=0.001,                     # standard weight decay 0.001
    fp16=False,                             # set to true for A100
    bf16=False,                             # set to true for A100
    max_grad_norm=0.3,                      # standard setting
    max_steps=-1,                           # needs to be -1, otherwise overrides epochs
    warmup_ratio=0.03,                      # standard warmup ratio
    group_by_length=True,                   # speeds up the training
    lr_scheduler_type="cosine",           # constant seems better than cosine
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,                # use our lora peft config
    dataset_text_field="text",
    max_seq_length=None,                    # no max sequence length
    tokenizer=tokenizer,                    # use the llama tokenizer
    args=training_arguments,                # use the training arguments
    packing=False,                          # don't need packing
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/home/laughing_bhabha/.local/lib/python3.8/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
/home/laughing_bhabha/.local/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2182 [00:00<?, ? examples/s]

Step,Training Loss
10,1.602800
20,1.662400
30,1.440000
40,1.904400
50,3.080300
60,1.248800
70,1.374300
80,1.252300
90,1.540600
100,2.313200


In [10]:
trainer.model.push_to_hub("mistral_b_SAS_finetuned_50_test")
tokenizer.push_to_hub("mistral_b_SAS_finetuned_50_test")

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ayman56/mistral_b_SAS_finetuned_50_test/commit/ced565c7db99b65392f5086dd899cd0c7dbf935d', commit_message='Upload tokenizer', commit_description='', oid='ced565c7db99b65392f5086dd899cd0c7dbf935d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

parquet_file = r'test_data.parquet'
df = pd.read_parquet(parquet_file, engine='pyarrow')
print(df.head())
pipe = pipeline(task="text-generation", model=new_model, tokenizer=tokenizer, max_length=1000)
results = []
def generate_script(instruction):
    result = pipe(f"[INST] {instruction} [/INST]")
    generated_text = result[0]['generated_text']
    cleaned_output = generated_text.replace(f"[INST] {instruction} [/INST]", "").strip()
    return instruction, cleaned_output

# Use ThreadPoolExecutor to parallelize script generation
with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust max_workers based on your hardware
    future_to_inst = {executor.submit(generate_script, inst): inst for inst in df['Annotation'].iloc[:100]}
    for future in tqdm(as_completed(future_to_inst), total=len(future_to_inst), desc="Generating Scripts"):
        instruction, script = future.result()
        results.append({"instruction": instruction, "generated_script": script})

# Convert results to DataFrame and save to Parquet
df_results = pd.DataFrame(results)
df_results.to_parquet('generated_scripts.parquet')

parquet_file = r'generated_scripts.parquet'
df=pd.read_parquet(parquet_file, engine = 'pyarrow')
df

In [1]:
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


# Specify the model and tokenizer name from the Hugging Face model hub
model_name = "ayman56/mistral_b_SAS_finetuned_50_test"
tokenizer_name = "ayman56/mistral_b_SAS_finetuned_50_test"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Load and preprocess your dataset
parquet_file = 'test_data.parquet'
df = pd.read_parquet(parquet_file, engine='pyarrow')

# Define a text generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)

# Define a function to generate scripts
def generate_script(instruction):
    result = pipe(f"[INST] {instruction} [/INST]")
    generated_text = result[0]['generated_text']
    cleaned_output = generated_text.replace(f"[INST] {instruction} [/INST]", "").strip()
    return instruction, cleaned_output

# Use ThreadPoolExecutor to parallelize script generation
results = []
with ThreadPoolExecutor(max_workers=5) as executor:
    future_to_inst = {executor.submit(generate_script, inst): inst for inst in df['Annotation'].iloc[:100]}
    for future in tqdm(as_completed(future_to_inst), total=len(future_to_inst), desc="Generating Scripts"):
        instruction, script = future.result()
        results.append({"instruction": instruction, "generated_script": script})

# Convert results to DataFrame and save to Parquet
df_results = pd.DataFrame(results)
df_results.to_parquet('generated_scripts.parquet')

# Load the generated scripts DataFrame
generated_scripts_file = 'generated_scripts.parquet'
df_generated = pd.read_parquet(generated_scripts_file, engine='pyarrow')
print(df_generated)


2024-04-11 05:56:52.158180: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-11 05:56:52.205680: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Generating Scripts:   0%|          | 0/100 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Generating Scripts:   0%|          | 0/100 [05:13<?, ?it/s]


KeyboardInterrupt: 

In [8]:
from transformers import pipeline

prompt = "Create a new data table called (sports) which has only three columns from the (cars) data set: make, type, and msrp. Keep only those observations where type is equal to (Sports) and msrp is greater than $100,000. Then, create another data table called (suv) which has the same three columns. Keep only those observations where type is equal to (SUV) and msrp is greater than $60,000. Finally, create a new data table called (expensive) by row-binding (sports) and (suv). Print (expensive) to see the results."

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])


[INST] Create a new data table called (sports) which has only three columns from the (cars) data set: make, type, and msrp. Keep only those observations where type is equal to (Sports) and msrp is greater than $100,000. Then, create another data table called (suv) which has the same three columns. Keep only those observations where type is equal to (SUV) and msrp is greater than $60,000. Finally, create a new data table called (expensive) by row-binding (sports) and (suv). Print (expensive) to see the results. [/INST] /*--------------------------------------------------*/
/* SAS Programming for R Users - code for exercises */
/* Copyright 2016 SAS Institute Inc.                */
/*--------------------------------------------------*/

/*SP4R03s04*/

/*Part A*/
data sp4r.sports;
   set sp4r.cars;
   keep make type msrp;
   where type='Sports' and msrp>100000;
run;

/*Part B*/
data sp4r.suv;
   set sp4r.cars;
   keep make type msrp;
   where type='SUV' and msrp>60000;
run;

/*Part C*/
da

In [7]:
from transformers import pipeline

prompt = "Create a new data table called (sports) which has only three columns from the (cars) data set: make, type, and msrp."

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])


[INST] Create a new data table called (sports) which has only three columns from the (cars) data set: make, type, and msrp. Keep only those observations where type is equal to (Sports) and msrp is greater than $100,000. Then, create another data table called (suv) which has the same three columns. Keep only those observations where type is equal to (SUV) and msrp is greater than $60,000. Finally, create a new data table called (expensive) by row-binding (sports) and (suv). Print (expensive) to see the results. [/INST] /*--------------------------------------------------*/
/* SAS Programming for R Users - code for exercises */
/* Copyright 2016 SAS Institute Inc.                */
/*--------------------------------------------------*/

/*SP4R03s05*/

/*Part A*/
proc sql;
   create table sp4r.sports as select
      make,
      type,
      msrp
   from sp4r.cars
   where type ='Sports' and msrp > 100000
   ;
quit;

/*Part B*/
proc sql;
   create table sp4r.suv as select
      make,
      

## Evaluation using GPT4 (Error)

In [ ]:
import csv
import openai
import sys
import os
from openai import AzureOpenAI
# Increase CSV field size limit to accommodate large fields
csv.field_size_limit(sys.maxsize)

# Configure OpenAI API
# openai.api_key = 'xxxxxxxxxxxxxxxxxxx'
client = AzureOpenAI(
    api_key = "96b0576933c646ed974b359811db4a8b",
    api_version="2024-02-01",
    azure_endpoint="https://labopenai.openai.azure.com",
    azure_deployment='gpt-4'
)

def generate_annotation_for_script(instruction, script):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages= [{"role": "user", "content": f"""GPT Scoring Prompt:
You are a helpful assistant that helps evaluate the quality of a SAS script response to a given instruction.
Answer by awarding a score between 0 and 10 to the response, where 0 means the response is completely 
inappropriate and 10 means the response is very good. A response that is acceptable should never be awarded less
than 6 out of 10.

Answer based on the following criteria:
1. Is the script syntactically correct?
2. Does the script accurately perform the task as per the instruction?
3. Is the script efficiently written, using SAS best practices?
4. Is the script clear and maintainable?

Output format : <score from 0 to 10>

Rate the response to the following instruction: {instruction}

SAS Script: {script}"""}],
            temperature=0.5,
            max_tokens=100,  # Adjust based on how lengthy we expect the responses to be
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Annotation could not be generated."